# Load libraries

In [1]:
#base
import re
import os
import pandas as pd

#webscraper
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib.request

#multiprocessing
import multiprocessing as mp

#slow the scraper down a little
import time

# setup selenium scraper

In [2]:
driver = webdriver.Chrome(executable_path="../dependencies/chromedriver")

In [3]:
url = "https://podcastluisteren.nl/pod/Maarten-van-Rossem-De-Podcast"
driver.get(url)

# Find podcast titles and download links

In [41]:
elements = driver.find_elements(By.XPATH, "//h4[@class='mt-1 text-left']")

In [42]:
titles = [element.text for element in elements]
titles = [title.replace("/", "-") for title in titles]


In [43]:
data = pd.DataFrame()
data["titles"] = titles

In [44]:
buttons = [element.find_element(By.XPATH, "../div/button") for element in elements]

In [45]:
date_duration = driver.find_elements(By.XPATH, "//h4[@class='text-left mb-4']")
date_duration = [element.text for element in date_duration]

In [55]:
data["date_and_duration"] = date_duration

In [47]:
def find_audio_path(button, audio_element):
    # Start stream of episode
    button.click()
    # Pause the stream, as we only need it loaded
    button.click()
    time.sleep(0.01)
    
    src = audio_element.get_attribute("src")
    return src

In [48]:
audio = driver.find_element(By.XPATH, "//audio")
sources = [find_audio_path(button, audio) for button in buttons]

In [49]:
data["sources"] = sources

In [ ]:
temp = data["date_and_duration"].str.split("|", n = 1, expand = True)
data["date"] = temp[0]
data["duration"] = temp[1]
data = data.drop(columns="date_and_duration")

In [84]:
data["date"] = pd.to_datetime(data['date'])

In [108]:
episode = data["titles"].str.findall("(?:#)(\d+)").str[0]
data["episode"] = episode

In [123]:
data["episode"] = data["episode"].fillna(-9999)
data["episode"] = data["episode"].astype(int)

In [115]:
data["mp3_path"] = data["titles"].transform(lambda title: f"../data/audio/{title}.mp3")

In [120]:
data["txt_path"] = data["titles"].transform(lambda title: f"../data/text (copy)/file:{title}.mp3.txt")

In [121]:
data

,titles,sources,date,duration,episode,mp3_path,txt_path
0,#311 - Maartens wens voor hij doodgaat,https://anchor.fm/s/21c734c4/podcast/play/6088...,2022-11-19,00:37:04,311,../data/audio/#311 - Maartens wens voor hij do...,../data/text (copy)/file:#311 - Maartens wens ...
1,#310 - Aanval op Kremlin,https://anchor.fm/s/21c734c4/podcast/play/6076...,2022-11-17,00:31:36,310,../data/audio/#310 - Aanval op Kremlin.mp3,../data/text (copy)/file:#310 - Aanval op Krem...
2,Maarten en Tom op RTL 4,https://anchor.fm/s/21c734c4/podcast/play/6069...,2022-11-16,00:09:31,NaN,../data/audio/Maarten en Tom op RTL 4.mp3,../data/text (copy)/file:Maarten en Tom op RTL...
3,#309 - VVD-implosie,https://anchor.fm/s/21c734c4/podcast/play/6064...,2022-11-15,00:36:56,309,../data/audio/#309 - VVD-implosie.mp3,../data/text (copy)/file:#309 - VVD-implosie.txt
4,#308 - Trump gaat eraan!,https://anchor.fm/s/21c734c4/podcast/play/6055...,2022-11-13,00:39:50,308,../data/audio/#308 - Trump gaat eraan!.mp3,../data/text (copy)/file:#308 - Trump gaat era...
...,...,...,...,...,...,...,...
320,#4 - Maarten van Rossem drinkt elke nacht bier,https://anchor.fm/s/21c734c4/podcast/play/1945...,2020-09-11,00:51:17,4,../data/audio/#4 - Maarten van Rossem drinkt e...,../data/text (copy)/file:#4 - Maarten van Ross...
321,#3 - Je moet strakke dames van middelbare leef...,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:32:10,3,../data/audio/#3 - Je moet strakke dames van m...,../data/text (copy)/file:#3 - Je moet strakke ...
322,#2 - Heerlijk! Een zomer zonder festivals,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:29:11,2,../data/audio/#2 - Heerlijk! Een zomer zonder ...,../data/text (copy)/file:#2 - Heerlijk! Een zo...
323,#1 - De terminale arrogantie van Wopke Hoekstra,https://anchor.fm/s/21c734c4/podcast/play/1485...,2020-06-07,00:25:14,1,../data/audio/#1 - De terminale arrogantie van...,../data/text (copy)/file:#1 - De terminale arr...


### Write data to file

In [124]:
data.to_pickle("data.pickle")

# Download the mp3 files

In [39]:
use_cores = mp.cpu_count()

In [40]:
def download_mp3(source, title):
    path = f"../data/audio/{title}.mp3"
    if os.path.exists(path):
        return
    urllib.request.urlretrieve(source, path)
    time.sleep(2)

In [41]:
pool = mp.Pool(use_cores)
result = pool.starmap(download_mp3, tuple(zip(sources, titles)))